In [ ]:
from safetensors import safe_open
import torch
import quiptools_cuda

In [ ]:
import grid

packed_abs_grid = grid.get_packed_abs_grid()

In [ ]:
tensors = {}
with safe_open("model.safetensors", framework="pt", device="cpu") as f:
   for key in f.keys():
       tensors[key] = f.get_tensor(key)

In [ ]:
for key in tensors.keys():
    if 'model.layers.0.mlp.down_proj' not in key:
        continue
    print(key)

In [ ]:
grid = set()

In [ ]:
for layer_idx in range(32):
    PREFIX = f'model.layers.{layer_idx}.mlp.down_proj.'

    print(PREFIX)
    
    Qidxs = tensors[f'{PREFIX}Qidxs']
    SU = tensors[f'{PREFIX}SU']
    SV = tensors[f'{PREFIX}SV'] * tensors[f'{PREFIX}Wscale']
    codebook_id = tensors[f'{PREFIX}codebook_id']
    assert codebook_id.item() == 7
    
    n, m = len(SU), len(SV)
    
    W = quiptools_cuda.decompress_packed_e8p(
        Qidxs.view(m // 16, n // 64, 8, 4).cuda(),
        packed_abs_grid.cuda(),
    )
    
    grid.update((tuple(a) for a in W.reshape(-1, 8).cpu().numpy()))
    
    print(len(grid), 2 ** 16)

    if len(grid) == 2 ** 16:
        break

In [ ]:
grid_by_idx = sorted(grid)
grid_tensor = torch.tensor(grid_by_idx)

In [ ]:
torch.save(grid_tensor, 'e8p_grid.pt')